# GRPM Dataset Builder 2.0 (LitVar2 Patch)

This notebook is designed to retrieve genetic polymorphism data from multiple sources. It uses the LitVar API to extract polymorphisms for each human gene within the LitVar2 database along with all associated PubMed Identifiers (PMIDs). These PMIDs are then employed as queries on PubMed to obtain MEDLINE data (parsed though the 'nbib' package). All collected data are ultimately consolidated into a single CSV file, known as the "GRPM Dataset", which serves as the primary source against which MeSH term queries can be launched to retrieve genes and polymorphisms associated with specific contexts.  

**Updates** : 

The GRPM Dataset available on Zenodo is a snapshot of [LitVar1](https://www.ncbi.nlm.nih.gov/CBBresearch/Lu/Demo/LitVar/help.html). LitVar1 is now <u>**deprecated**</u> and has been fully replaced by [LitVar2](https://www.ncbi.nlm.nih.gov/research/litvar2/). This module 
([Dataset Builder](https://github.com/johndef64/GRPM_system/blob/main/GRPM_01_dataset_builder.ipynb)) has been updated to retrieve data from LitVar2. The subsequent modules in the pipeline remain functional and can be tested using the original version of the GRPM Dataset available on Zenodo.

In [1]:
#Only for Google Colab
import ast
import os
import sys

# @markdown Run in Colab virtual machine by default

# @markdown to run in google drive set:
import_mydrive = False #@param {type:"boolean"}

if 'google.colab' in sys.modules:
    !pip install nbib
    !pip install biopython

    if import_mydrive:
        from google.colab import drive
        drive.mount('/content/drive')
        if os.path.exists('/content/drive/MyDrive/grpm_system/'):
            %cd /content/drive/MyDrive/grpm_system/
        else:
            %mkdir /content/drive/MyDrive/grpm_system/
            %cd /content/drive/MyDrive/grpm_system/
    else:
        if os.path.exists('/content/grpm_system/'):
            %cd /content/grpm_system/
        else:
            %mkdir /content/grpm_system/
            %cd /content/grpm_system/

current_directory = os.getcwd()
print("Current working directory:", current_directory)

Current working directory: G:\Altri computer\Horizon\horizon_workspace\projects\bioinformatics\semantics\GRPM\GRPM_dev


# Get required data from Zenodo

In [2]:
# Get Required datasets from Zenodo Repository
#https://zenodo.org/record/8205724  DOI: 10.5281/zenodo.8205724

import os
import io
import requests
import zipfile

def get_and_extract(file, dir = os.getcwd()):
    url = "https://zenodo.org/record/8205724/files/"+file+".zip?download=1"
    zip_file_name = file+".zip"
    extracted_folder_name = dir

    # Download the ZIP file
    response = requests.get(url)

    if response.status_code == 200:
        # Extract the ZIP contents
        with io.BytesIO(response.content) as zip_buffer:
            with zipfile.ZipFile(zip_buffer, 'r') as zip_ref:
                zip_ref.extractall(extracted_folder_name)
        print(f"ZIP file '{zip_file_name}' extracted to '{extracted_folder_name}' successfully.")
    else:
        print("Failed to download the ZIP file.")

if not os.path.exists('human_geness_repo'):
    get_and_extract('human_genes_repo')

ZIP file 'human_genes_repo.zip' extracted to 'G:\Altri computer\Horizon\horizon_workspace\projects\bioinformatics\semantics\GRPM\GRPM_dev' successfully.


# Import Requirements and Functions

In [23]:
#@Import Requirements and Functions
import importlib

def simple_bool(message):
    choose = input(message+" (y/n): ").lower()
    your_bool = choose in ["y", "yes"]
    return your_bool

def check_and_install_module(module_name):
    try:
        # Check if the module is already installed
        importlib.import_module(module_name)
        print(f"The module '{module_name}' is already installed.")
    except ImportError:
        # If the module is not installed, try installing it
        x = simple_bool(
            "\n" + module_name + "  module is not installed.\nwould you like to install it?")
        if x:
            import subprocess
            subprocess.check_call(["pip", "install", module_name])
            print(f"The module '{module_name}' was installed correctly.")
        else:
            pass

check_and_install_module('nbib')
check_and_install_module('requests')

#Import Modules
import os
import sys
import json
import nbib
import pandas as pd
import requests as rq
import matplotlib.pyplot as plt
from datetime import datetime
from bs4 import BeautifulSoup
from io import StringIO
from Bio import Entrez
import time

Entrez.email = "your_email@example.com"

request_counter = 0
gene_counter = 0

# Job Functions 
def transform_string(string):
    string = string.replace("\n",", ")
    string = string.replace("\'","\"")
    string = string.replace('\"\"', '\"')
    string = string.replace('p.\"','p.')
    string = string.replace('c.\"','c.')
    string = string.replace('g.\"','g.')
    string = string.replace('\">','>')
    string = string.replace('.C\"204','.C204') #<= if stucked, look for bugs like this into text
    return string

def query_build(pmid_list):
    query = "+OR+".join(pmid_list)
    return query

def build_pubmed_query(pmid_l, limit = 1300):
    query = []

    if len(pmid_l)<=limit:
        pmid_l01 = pmid_l
        query = [query_build(pmid_l01)]

    if limit<len(pmid_l)<=limit*2:
        j = len(pmid_l)//2
        pmid_l01 = pmid_l[:j]
        pmid_l02 = pmid_l[j:]
        query = [query_build(pmid_l01),
                 query_build(pmid_l02)]

    if limit*2<len(pmid_l)<=limit*3:
        j = len(pmid_l)//3
        pmid_l01 = pmid_l[:j]
        pmid_l02 = pmid_l[j:j*2]
        pmid_l03 = pmid_l[j*2:]
        query = [query_build(pmid_l01),
                 query_build(pmid_l02),
                 query_build(pmid_l03)]

    if limit*3<len(pmid_l)<=limit*4:
        j = len(pmid_l)//4
        pmid_l01 = pmid_l[:j]
        pmid_l02 = pmid_l[j:j*2]
        pmid_l03 = pmid_l[j*2:j*3]
        pmid_l04 = pmid_l[j*3:]
        query = [query_build(pmid_l01),
                 query_build(pmid_l02),
                 query_build(pmid_l03),
                 query_build(pmid_l04)]

    if limit*4<len(pmid_l)<=limit*5:
        j = len(pmid_l)//5
        pmid_l01 = pmid_l[:j]
        pmid_l02 = pmid_l[j:j*2]
        pmid_l03 = pmid_l[j*2:j*3]
        pmid_l04 = pmid_l[j*3:j*4]
        pmid_l05 = pmid_l[j*4:]
        query = [query_build(pmid_l01),
                 query_build(pmid_l02),
                 query_build(pmid_l03),
                 query_build(pmid_l04),
                 query_build(pmid_l05)]

    if limit*5<len(pmid_l)<=limit*6:
        j = len(pmid_l)//6
        pmid_l01 = pmid_l[:j]
        pmid_l02 = pmid_l[j:j*2]
        pmid_l03 = pmid_l[j*2:j*3]
        pmid_l04 = pmid_l[j*3:j*4]
        pmid_l05 = pmid_l[j*4:j*5]
        pmid_l06 = pmid_l[j*5:]
        query = [query_build(pmid_l01),
                 query_build(pmid_l02),
                 query_build(pmid_l03),
                 query_build(pmid_l04),
                 query_build(pmid_l05),
                 query_build(pmid_l06)]

    if limit*6<len(pmid_l)<=limit*7:
        j = len(pmid_l)//7
        pmid_l01 = pmid_l[:j]
        pmid_l02 = pmid_l[j:j*2]
        pmid_l03 = pmid_l[j*2:j*3]
        pmid_l04 = pmid_l[j*3:j*4]
        pmid_l05 = pmid_l[j*4:j*5]
        pmid_l06 = pmid_l[j*5:j*6]
        pmid_l07 = pmid_l[j*6:]
        query = [query_build(pmid_l01),
                 query_build(pmid_l02),
                 query_build(pmid_l03),
                 query_build(pmid_l04),
                 query_build(pmid_l05),
                 query_build(pmid_l06),
                 query_build(pmid_l07)]
    return query


def plot_data(df_count_sort, timestamp):
    x1 = df_count_sort['mesh'].head(30)
    y1 = df_count_sort['pmid-unique'].head(30)
    plt.figure(figsize=(5, 8))
    plt.title('Scatter Plot: '+gene+' pmid-mesh (total)', loc='center',pad=10)
    plt.scatter(y1, x1)
    plt.gca().invert_yaxis()
    #plt.yticks(rotation=90)
    plt.tick_params(axis='x', which='both', top=True, bottom=False, labeltop=True, labelbottom=False)
    plt.xlabel('pmid count', position=(0.5, 1.08))
    ax = plt.gca()
    ax.xaxis.set_label_position('top')
    plt.savefig(db_path+'/'+gene+'_mesh_plot_'+timestamp+'_total.png',dpi=120, bbox_inches = "tight")
    plt.close()


def get_studytype_data(ref):
    dfbib = pd.DataFrame(ref)
    dfbib.pubmed_id = dfbib.pubmed_id.astype('str')
    if 'publication_types' in dfbib.columns and len(dfbib['publication_types'])>1:
        dfbib_studyty = dfbib[['pubmed_id','publication_types']].dropna().reset_index(drop=True)

        #PMID-Studytype table build:
        df_studytype = []
        for i in range(len(dfbib_studyty)):
            for studytype in dfbib_studyty['publication_types'][i]:
                out = dfbib_studyty['pubmed_id'][i], studytype
                df_studytype.append(out)
        STUDYT = pd.DataFrame(df_studytype).rename(columns={0: 'pmids',1:'study_type'})
        mask_st = STUDYT['study_type'].str.contains('Research Support|Journal Article')
        STUDYTless = STUDYT[~mask_st].reset_index(drop=True)

        mask_lessing = STUDYT['pmids'].isin(STUDYTless['pmids'])
        STUDYTdiff = STUDYT[~mask_lessing].reset_index(drop=True)
        STUDYTdiff['study_type2'] = 'Unknown'
        STUDYTdiff = STUDYTdiff[['pmids','study_type2']].rename(columns={'study_type2':'study_type'}).drop_duplicates().reset_index(drop=True)
        STUDYTconcat = pd.concat([STUDYTless, STUDYTdiff], ignore_index=True)

        STUDYTconcat.to_csv(db_path+'/'+gene+'_lit1pmid_studytype.csv')
    else:
        print(gene+' no publication_types in nbib')
        pass

The module 'nbib' is already installed.
The module 'requests' is already installed.


# Whole genome forecast

In [2]:
#Statistics based on 150 random genes:
time_sleep = 0.4
runtime_gene = 6.36 #sec/gene
genes_hour = 566 #genes/hour
request_counter_gene = 4.25 #request/gene (with base sleep (0.4))
sleep_request_base = 0.4 #time sleep each request
sleep_request_overnight_plus = 1.1 # for an overnight job

print('Forecast:')
max_genes = int(10000/request_counter_gene)
table_size_db_gene = 0.496 #MB
table_size_gene = 0.397 #MB
png_size_db_gene = 0.47 #KB

#Forecast:
genes = pd.read_csv('human_genes_repo/H_GENES_proteincoding_genes.csv')
ngenes = len(genes)#gene_range
nruntime = ngenes * runtime_gene
#print('runtime, '+str(ngenes), nruntime)
nrequest_counter = ngenes * request_counter_gene

tempo_ore = round(nruntime/3600, 2)
tempo_ore_overnight = round((nruntime+(sleep_request_overnight_plus*ngenes))/3600, 2)

print('max genes/day= ',max_genes)
print('for',str(int(ngenes)),'genes:')
print('    request counter =', nrequest_counter,'requests')
print('    whole genome runtime =', tempo_ore,'hours')
print('    whole genome runtime overnight =', tempo_ore_overnight)

db_table_size = ngenes * table_size_gene
print('    db table size', round(db_table_size,2),'MB')

Forecast:
max genes/day=  2352
for 19383 genes:
    request counter = 82377.75 requests
    whole genome runtime = 34.24 hours
    whole genome runtime overnight = 40.17
    db table size 7695.05 MB


# Load Human Genes

In [3]:
# Load Human Gene list ---------------------------------
protein_coding_genes = pd.read_csv('human_genes_repo/H_GENES_proteincoding_genes.csv')
IG_TR_genes          = pd.read_csv('human_genes_repo/H_GENES_IGTR_genes.csv')
RNA_genes            = pd.read_csv('human_genes_repo/H_GENES_RNA_genes.csv')
pseudo_genes         = pd.read_csv('human_genes_repo/H_GENES_pseudo_genes.csv')
misc_genes           = pd.read_csv('human_genes_repo/H_GENES_misc_genes.csv')

# create gene lists:
protein_coding_genes_list = protein_coding_genes['Gene name'].dropna().tolist()
rna_genes_list = RNA_genes['Gene name'].dropna().tolist()
pseudo_genes_list = pseudo_genes['Gene name'].dropna().tolist()


# Split job packages:----------------------------------

# (1) protein coding genes:
gene_range = int(len(protein_coding_genes_list)/18)
genes = [protein_coding_genes_list[i * gene_range : (i + 1) * gene_range] for i in range(0, 18)]
pcg_chunks = genes[:18]

# (2) RNA genes:
rna_gene_range = int(len(rna_genes_list)/5)
genes = [rna_genes_list[i * rna_gene_range : (i + 1) * rna_gene_range] for i in range(0, 8)]
rna_chunks = genes[:5]

# (3) pseudo genes:
pseudo_gene_range = int(len(pseudo_genes_list)/2)
genes = [rna_genes_list[i * pseudo_gene_range : (i + 1) * pseudo_gene_range] for i in range(0, 8)]
pseudo_chunks = genes[:2]

print('protein_coding_genes',len(protein_coding_genes['Gene name'].dropna()),
      '\nIG_TR_genes',len(IG_TR_genes['Gene name'].dropna()),
      '\nRNA_genes',len(RNA_genes['Gene name'].dropna()),
      '\npseudo_genes',len(pseudo_genes['Gene name'].dropna()),
      '\nmisc_genes',len(misc_genes['Gene name'].dropna()))

print('\nrecommended job lenght for pcg:',int(len(protein_coding_genes_list)/18))

protein_coding_genes 19318 
IG_TR_genes 641 
RNA_genes 11452 
pseudo_genes 9866 
misc_genes 22

recommended job lenght for pcg: 1073


# Set options and import building dataset

In [10]:
# set db:--------------------------------
db_tag = 'pcg_2024_lit2'
    # 'pcg'    = protein coding genes = grpm_db
    # 'rna'    = rna genes            = grpm_db_rna
    # 'pseudo' = pseudogenes          = grpm_db_pseudo

db_name = 'grpm_db_'+db_tag
db_path = 'grpm_dataset/'+db_name

if not os.path.exists(db_path):
    os.makedirs(db_path)
#------------------------------------------------


#import checkpoint datasets:
time_a = datetime.now()
if os.path.isfile(db_path+'/grpm_table_output.csv'):
    complete_df = pd.read_csv(db_path+'/grpm_table_output.csv',index_col=0)
    restart = True
else:
    complete_df = pd.DataFrame()
    restart = False

if os.path.isfile(db_path+'/complete_nbibtable.csv'):
    complete_nbibtable = pd.read_csv(db_path+'/complete_nbibtable.csv',index_col=0)
else:
    complete_nbibtable = pd.DataFrame()
time_b = datetime.now()

print('time load',time_b-time_a)

## check saved data:
if os.path.isfile(db_path+'/grpm_table_output.csv'):
    gene_db_count =  complete_df.gene.nunique()
    print('complete_df gene count:',gene_db_count,'on', len(protein_coding_genes_list))
    if gene_db_count >= 15519:
        print('grpm db already contains all available genes on litvar1')

    print('\ngrpm_table_output.csv size'  ,round(os.path.getsize(db_path+'/grpm_table_output.csv')/(1024*1024),3),'MB')
    print('complete_nbibtable.csv size',round(os.path.getsize(db_path+'/complete_nbibtable.csv')/(1024*1024),3),'MB')
    print('memory_usage_complete_df'     ,round(complete_df.memory_usage().sum()/(1024*1024),3))
    print('memory_usage_complete_nbib_df',round(complete_nbibtable.memory_usage().sum()/(1024*1024),3))
else:
    print('empty dataset')
    print('empty dataset')

time load 0:00:00.043998
complete_df gene count: 9 on 19318

grpm_table_output.csv size 0.027 MB
complete_nbibtable.csv size 0.252 MB
memory_usage_complete_df 0.023
memory_usage_complete_nbib_df 0.009


# Run Job

## Set gene-range for this job

In [11]:
# Set the gene list ----------------

#1. Set gene-range [whole genome build]
    # pcg_chunks    [0:17]
    # rna_chunks    [0:4]
    # pseudo_chunks [0:1]
gene_chunk = pcg_chunks[0]

# run a sample?
run_sample = simple_bool('Do you want to run a chunk sample for testing?')
if run_sample:
    sample_size = int(input('sample size? \nnum:'))


#2. place here your custom gene list [custom build]
if not run_sample:
    custom_genes = ['APOA1', 'FFC1', 'ERH', 'USP53']
    custom_list = simple_bool('Do you want to run the custom gene list?')
else:
    custom_list = False

#if stucked, store skipped_genes and run custom list later:
skipped_genes =  []

# Set save interval checkpoint frequency
checkpoint = 5  #genes

## Run Job

In [24]:
# set options:--------------------------------
save_plot           = True
save_studytype_data = False
save_accessory_data = False

import ast
# Run Job -------------------------

if run_sample:
    genes = pd.Series(gene_chunk).sample(sample_size).reset_index(drop=True)
    genes = genes.to_list()
    restart = False
else:
    genes = pd.Series(gene_chunk)
    genes = genes.to_list()

if restart:
    restart_from = complete_df.gene.nunique()
    gene_start = restart_from
    print('search restarted from '+str(restart_from))
else:
    gene_start = 0

if custom_list:
    genes = custom_genes
    gene_start = 0

#=========================================================================


#=========================================================================
time_start = datetime.now()
print('Start at ',time_start)

# get gene data
for gene in genes[gene_start:]:

    #LitVar2 "Variants for Gene" API request
    if request_counter > 9950:
        print('Request limit reached. Wait \'till tomorrow!')
        pass
    time_alpha = datetime.now()
    url = "https://www.ncbi.nlm.nih.gov/research/litvar2-api/variant/search/gene/" + gene
    response = (rq.get(url)).text

    # parsing output in JSON
    
    data= "[" + transform_string(response) + "]"
    
    #Create Dataframe
    df = pd.read_json(StringIO(data))
    if 'rsid' in df.columns and len(df.rsid)>1:
        # creare un df senza i clingen.
        dfb = df[['_id','pmids_count','rsid']]
        dfa = dfb[~dfb['_id'].str.contains('@CA')].drop_duplicates().reset_index(drop=True)
        dfn = dfa.dropna(subset=['rsid'])

        #Statistics
        #handle = Entrez.esearch(db="snp", term=gene)
        #record = Entrez.read(handle)
        #request_counter += 1

        NCBI_dbSNP = 'na' #record["Count"]
        lit2_variant = len(dfa['_id'].drop_duplicates())
        lit2_variant_norsid = len(dfa.loc[df['rsid'].isna()])
        lit2_rsid = len(dfn.rsid.drop_duplicates())

        # remove rs with pmid_count = 1
        df2 = dfn.loc[df.pmids_count !=1]#.reset_index(drop=True)
        lit2_rsid_f = len(df2)

        # accessory data
        dfsort = df.sort_values(by='pmids_count',ascending=False).reset_index(drop=True)
        df2sort = df2.sort_values(by='pmids_count',ascending=False).reset_index(drop=True)
      

        for rsid in df2.rsid[0:1]:
            url="https://www.ncbi.nlm.nih.gov/research/litvar2-api/variant/get/litvar@"+rsid+"%23%23/publications"
            response = (rq.get(url)).text
            response_dict = ast.literal_eval(response)
            #rspost = pd.DataFrame(response_dict)
            rspost = pd.DataFrame({key: pd.Series(value) for key, value in response_dict.items()})
            
            #Display-------------------------------------------------------
            dfrspost = pd.DataFrame(rspost)
            if 'pmids' in dfrspost.columns and len(dfrspost.pmids_count)>1:
            #if 'rsid' in dfrspost.columns and len(dfrspost.rsid)>1:
                #lit1_rsid = len(dfrspost.rsid)
                lit1_rsid = 0
                #lit2_rsid = len(df)
    
                # Creating the simple list [rsid-pmid]========================
    
                rsidpmid = dfrspost[['pmids']].copy()
                rsidpmid['rsid'] = str(rsid)
                rsidpmid['pmids'] = rsidpmid['pmids'].astype('str')
                
                #report data:
                lit1_rsid_pmid = len(rsidpmid)
                lit1_pmid = len(rsidpmid.drop_duplicates(subset='pmids'))
    
    
                ####[MODULE: groupby.describe]
                # applicare groupby ad rsidpmid per avere tabella pmid count
                rsidpmidcount = rsidpmid.groupby('rsid').describe().reset_index()
                rsidpmidcount.columns = rsidpmidcount.columns.to_flat_index()
            

                #replace column names
                new_column_names = ['rsid', 'pmid_count', 'pmid_unique','pmid_top','pmid_freq']
                rsidpmidcount.columns = new_column_names
                rsidpmidcountf = rsidpmidcount[['rsid','pmid_unique']]
    
                #report data:-------------------------------------------------------------
                lit1_rsid_f = len(rsidpmidcountf[rsidpmidcountf.pmid_unique!=1])
                lit1_rsid_m = len(rsidpmidcountf[rsidpmidcountf.pmid_unique==1])
    
                rsidpmidcountfsort = rsidpmidcountf.sort_values('pmid_unique',ascending=False).reset_index(drop=True)
    
    
                #Filter pmid for rsid with pmid>1------------------------------------------
                outless = rsidpmidcountfsort[rsidpmidcountfsort.pmid_unique>1]
                #creare una mask isin su rsidpmid con outless.rsid
                mask = rsidpmid['rsid'].isin(outless.rsid)
                rsidpmidless = rsidpmid[mask]
                lit1_pmid_f = len(rsidpmidless.pmids.drop_duplicates())
    
    
                # PubMed query Build:=====================================================
                ### two input alternatives (total LitVar1 and LitVar>1)
                #Total
                pmid_l = rsidpmid.pmids.drop_duplicates().tolist()

                ##Define list of queries for PubMed:
                query = build_pubmed_query(pmid_l, limit = 1300)
                
    
                # Merging requests for the queries
                ### carefull: high runtime
    
                time1 = datetime.now()
                pages = ((len(pmid_l)//200)+1)+1
                if len(pmid_l) % 200 == 0:
                    pages = pages -1
                fullnbib = str()
                for d in query:
                    for i in range(1,pages):
                        page = str(i)
                        url = 'https://pubmed.ncbi.nlm.nih.gov/?term=' + d + '&format=pubmed&size=200&page='+ page
                        output = rq.get(url)
                        html = output.text
                        soup = BeautifulSoup(html, features="html.parser")
                        for script in soup(["script", "style"]):
                            script.extract()
                        text = soup.get_text()
                        postString = text.split("\n\n\n\n\n\n\n\n\n\n",2)[2]
                        nbib01 = postString.replace('\n\n','')
                        fullnbib += nbib01
                        request_counter += pages
                        time.sleep(1.5)
    
                time2 = datetime.now()
                timestamp = time2.strftime('%Y%m%d%H%M%S')
                #^print('finish at:', datetime.now())
                runtime = time2-time1
                duration = str(runtime).split('.')[0]
                hours, minutes, seconds = duration.split(':')
                compact_duration = '{}:{}:{}'.format(hours, minutes, seconds)
    
    
                # nbib parsing:
                timea = datetime.now()
                ref = nbib.read(fullnbib)
                dfbib = pd.DataFrame(ref)
                if 'descriptors' in dfbib.columns and len(dfbib['descriptors'])>1:
                    dfbibdes = dfbib[['pubmed_id','descriptors']].dropna().reset_index(drop=True)
                    nbib_objects = len(dfbib)
                    nbib_objects_withdescriptors = len(dfbibdes)
                    #print('nibib objects:',nbib_objects)
                    #print('nibib objects with descriptors:',len(dfbibdes))
                    timeb = datetime.now()
                    #print('runtime:', timeb-timea)
    
                    #Statistics:
                    pubmed_pmid_query = len(pmid_l)
                    pubmed_pmid_nbib = len(dfbib.pubmed_id.drop_duplicates())
                    pubmed_pmid_nbib_yesmesh = len(dfbibdes.pubmed_id.drop_duplicates())
                    pubmed_pmid_nbib_nomesh = len(dfbib.pubmed_id.drop_duplicates())-len(dfbibdes.pubmed_id.drop_duplicates())
    
                    # refine MESH
                    dfr = []
                    for i in range(len(dfbibdes)):
                        for mesh in dfbibdes['descriptors'][i]:
                            out = dfbibdes['pubmed_id'][i], mesh
                            dfr.append(out)
                    MESH = pd.DataFrame(dfr).rename(columns={0: 'pmids',1:'mesh'})
    
                    # dataframe parsing splitting three fields
                    MESHsplit =[]
                    for i in range(len(MESH)):
                        mg = MESH.mesh[i].get('descriptor')
                        mg2 = MESH.mesh[i].get('qualifier')
                        mg3 = MESH.mesh[i].get('major')
                        mgg = MESH.pmids[i], mg, mg2, mg3
                        MESHsplit.append(mgg)
    
                    dfmesh = pd.DataFrame(MESHsplit).rename(columns={0: 'pmids',1:'mesh',2:'qualifier',3:'major'}).drop_duplicates()
    
                    #statistics
                    pubmed_pmidmesh = len(dfmesh[['pmids','mesh']].drop_duplicates())
                    pubmed_mesh_qualifier_major = len(MESH.mesh.drop_duplicates())
                    pubmed_mesh = len(dfmesh.mesh.drop_duplicates())
    
                    pmidmesh = dfmesh[['pmids','mesh']].drop_duplicates()
                    pmidmesh['pmids'] = pmidmesh['pmids'].astype(str) #convert pmid type in str
    
    
                    #Analyze enrichment with groupby.describe method-------------------------------
                    #Add rsid coulmn con merge
                    rspmidmesh_merge = pd.merge(pmidmesh, rsidpmid, on= 'pmids', how='inner').drop_duplicates().reindex(columns=['pmids', 'rsid', 'mesh'])
                    #rspmidmesh_merge['pmids'] = rspmidmesh_merge['pmids'].astype(str)
    
                    ### groupby.describe analysis by mesh
                    meshrspmidmerge_count = rspmidmesh_merge.groupby('mesh').describe().reset_index()
                    meshrspmidmerge_count.columns = meshrspmidmerge_count.columns.to_flat_index()
                    #to handle generate df.groupby.describe, convert Multicolumn to single column
                    #https://datascientyst.com/flatten-multiindex-in-pandas/
                    new_column_names = ['mesh', 'pmid-count', 'pmid-unique','pmid-top','pmid-freq','rsid-count', 'rsid-unique','rsid-top','rsid-freq']
                    meshrspmidmerge_count.columns = new_column_names
    
                    meshrspmidmerge_count_short = meshrspmidmerge_count[['mesh','pmid-unique','rsid-unique']]
                    #pmidmeshintmerge2meshlesssort = pmidmeshintmerge2meshless.sort_values(by='pmid-unique',ascending=False).reset_index(drop=True)
    
                    # add frequency
                    totalpmid_count = len(pmidmesh.pmids.drop_duplicates())
                    meshrspmidmerge_count_short_freq = meshrspmidmerge_count_short.copy()
                    meshb_frq = meshrspmidmerge_count_short_freq.loc[:,'pmid-unique'].astype(float)/totalpmid_count
                    meshrspmidmerge_count_short_freq.loc[:,'mesh frequency'] = round(meshb_frq,3)#*100
                    meshrspmidmerge_count_short_freq_sort = meshrspmidmerge_count_short_freq.sort_values(by='pmid-unique',ascending=False).reset_index(drop=True)
    
                    top10mesh_all = meshrspmidmerge_count_short_freq_sort['mesh'][:10].tolist()
                    #display(meshrspmidmerge_count_short_freq_sort.head(20))
    
                    ### groupby.describe analysis by rsid------------------
                    rspmidmeshmerge_count = rspmidmesh_merge.groupby('rsid').describe().reset_index()
                    rspmidmeshmerge_count.columns = rspmidmeshmerge_count.columns.to_flat_index()
                    new_column_names = ['rsid', 'pmid-count', 'pmid-unique','pmid-top','pmid-freq','mesh-count', 'mesh-unique','mesh-top','mesh-freq']
                    rspmidmeshmerge_count.columns = new_column_names
    
                    rsid_pmid10 = len(rspmidmeshmerge_count[rspmidmeshmerge_count['pmid-unique']>10])
                    rsid_pmid50 = len(rspmidmeshmerge_count[rspmidmeshmerge_count['pmid-unique']>50])
                    rsid_pmid100 = len(rspmidmeshmerge_count[rspmidmeshmerge_count['pmid-unique']>100])
    
                    rspmidmeshmerge_count_short = rspmidmeshmerge_count[['rsid','pmid-unique','mesh-unique']]
                    rspmidmeshmerge_count_short_sort = rspmidmeshmerge_count_short.sort_values(by='pmid-unique', ascending= False).reset_index(drop=True)
                    top10rsid_all = rspmidmeshmerge_count_short_sort['rsid'].iloc[:10].tolist()

                    # create a scatter plot-----------------------------------------
                    if save_plot:
                        plot_data(meshrspmidmerge_count_short_freq_sort, timestamp)
 
                    # GET STUDY TYPE from NBIB----------------------------------------
                    if save_studytype_data:
                        get_studytype_data(ref)
    
    #SAVE TABLES-----------------------------------------------------------
                    timestamp = time2.strftime('%Y%m%d%H%M%S')
                    # save accessory data:
                    if save_accessory_data:
                        dfsort[["_id","rsid","pmids_count"]].to_csv(db_path+'/'+gene+'_litvar2_variants4gene.csv')
                        rsidpmid.to_csv(db_path+'/'+gene+'_litvar1_rsids2pmids.csv') #lit1 [rsid-pmid]
                        #rsidpmidcountfsort #lit1 pmid count
    
                        meshrspmidmerge_count_short_freq_sort.to_csv(db_path+'/'+gene+'_mesh_pmidrsid_count.csv')
    
                    #complete_df with concat:
                    #import gene-rsidpmidmesh and gene-rsidpmid
                    dfmesh['pmids'] = dfmesh['pmids'].astype(str)
                    rsidpmid['pmids'] = rsidpmid['pmids'].astype(str)
    
                    # add a rsid-merger to dfmesh
                    gene_rsidpmidmesh = pd.merge(rsidpmid, dfmesh, on='pmids')
                    gene_rsidpmidmesh['gene'] = gene
    
                    gene_df = pd.DataFrame(gene_rsidpmidmesh)
                    complete_df = pd.concat([complete_df, gene_rsidpmidmesh])
    
                    #complete_nbibtable with concat:
                    dfbib['gene'] = gene
                    complete_nbibtable = pd.concat([complete_nbibtable, dfbib])
                    #pyperclip.copy(str(dfbib.columns.to_list()))
    
                    # save checkpoint----------------------
                    if genes.index(gene) > 1 and genes.index(gene) % checkpoint == 0:
                        complete_df = complete_df.reindex(columns=['gene','rsid', 'pmids', 'mesh', 'qualifier', 'major'])
                        complete_df.to_csv(db_path+'/grpm_table_output.csv')
    
                        complete_nbibtable = complete_nbibtable.reindex(columns=['gene','pubmed_id', 'citation_owner', 'nlm_status', 'last_revision_date', 'electronic_issn', 'linking_issn', 'journal_volume', 'journal_issue', 'publication_date', 'title', 'abstract', 'authors', 'language', 'grants', 'publication_types', 'electronic_publication_date', 'place_of_publication', 'journal_abbreviated', 'journal', 'nlm_journal_id', 'descriptors', 'pmcid', 'keywords', 'conflict_of_interest', 'received_time', 'revised_time', 'accepted_time', 'pubmed_time', 'medline_time', 'entrez_time', 'pii', 'doi', 'publication_status', 'print_issn', 'pages'])
                        complete_nbibtable.to_csv(db_path+'/complete_nbibtable.csv')
                        print("saved checkpoint")
                    else:
                        pass
    
    
    #REPORT-----------------------------------------------------------------
                    time_omega = datetime.now()
                    full_runtime = time_omega - time_alpha
                    #^print('total runtime:', full_runtime)
                    print(gene + '_runtime:', full_runtime)
                    nbib_seconds = runtime.total_seconds()
                    total_seconds = full_runtime.total_seconds()
                    full_runtime_str = str(full_runtime).split('.')[0]
    
                    report = {'ncbi_dbsnp': NCBI_dbSNP,
                              'lit2_variant': lit2_variant,
                              'lit2_variant_norsid': lit2_variant_norsid,
                              'lit2_rsid': lit2_rsid,
                              'lit2_rsid_plus1': lit2_rsid_f,
                              'lit1_rsid': lit1_rsid,
                              #'lit1_raw_pmid': lit1_raw_pmid,
                              #'lit1_rsid_pmid': lit1_rsid_pmid,
                              'lit1_rsid_pmid_plus1': lit1_rsid_f,
                              #lit1_rsid_pmid=1': lit1_rsid_m,
                              'lit1_pmid': lit1_pmid,
                              'lit1_pmid_pmid_plus1': lit1_pmid_f,
                              'pubmed_pmid_query': pubmed_pmid_query,
                              'nbib_objects': nbib_objects,
                              'nbib_objects_withdescriptors': nbib_objects_withdescriptors,
                              'pubmed_pmid': pubmed_pmid_nbib,
                              'pubmed_pmid_withmesh': pubmed_pmid_nbib_yesmesh,
                              #'pubmed_pmid_nomesh':pubmed_pmid_nbib_nomesh,
                              'pubmed_pmidmesh': pubmed_pmidmesh,
                              'pubmed_mesh_qualifier_major': pubmed_mesh_qualifier_major,
                              'pubmed_mesh': pubmed_mesh,
                              'rsid_pmid10': rsid_pmid10,
                              'rsid_pmid50': rsid_pmid50,
                              'rsid_pmid100': rsid_pmid100,
                              'top10mesh_all': str(top10mesh_all),
                              'top10rsid_all': str(top10rsid_all),
                              'pubmed_runtime': duration,
                              'total_runtime': full_runtime_str,
                              'time_stamp': time2
                              }
    
                    df_report = pd.DataFrame(report, index=[gene]).transpose()
    
                    # generate fist report.csv
                    if os.path.isfile(db_path+'/GRPM_report.csv'):
                        dfL = pd.read_csv(db_path+'/GRPM_report.csv', index_col=0)
                        dfL = pd.concat([dfL, df_report], axis=1)
                        dfL.to_csv(db_path+'/GRPM_report.csv')
                    else:
                        df_report.to_csv(db_path+'/GRPM_report.csv')  # solo la prima volta
    
                    #Update gene values
                    GRPM_report = pd.read_csv(db_path+'/GRPM_report.csv', index_col=0)
                    if gene + '.1' in GRPM_report.columns:
                        GRPM_report = GRPM_report.drop(columns=gene)
                        GRPM_report = GRPM_report.rename(columns={gene + '.1': gene})
                        GRPM_report.to_csv(db_path+'/GRPM_report.csv')
                        print(gene,'already in db')
    
                else:
                    print(gene + ' no descriptors in nbib')
                    time.sleep(0.8)
                    pass
            else:
                print(gene + ' no results on litvar2 rsid2pmids')
                time.sleep(0.8)
                pass
    else:
        print(gene + ' no results on litvar2 gene2pmidcount')
        pass

    if request_counter > 9000:
        dada = 2
        #print('Allert! Reaching pubmed request limit')
    if request_counter > 9950:
        #print('Request limit reached. Wait \'till tomorrow!')
        time_finish = datetime.now()
        time_batch = time_finish - time_start
        time_batch_str = str(time_batch).split('.')[0]
        #print('time batch:', time_batch_str)
        #break

complete_df = complete_df.reindex(columns=['gene','rsid', 'pmids', 'mesh', 'qualifier', 'major'])
complete_df.to_csv(db_path+'/grpm_table_output.csv')

complete_nbibtable = complete_nbibtable.reindex(columns=['gene','pubmed_id', 'citation_owner', 'nlm_status', 'last_revision_date', 'electronic_issn', 'linking_issn', 'journal_volume', 'journal_issue', 'publication_date', 'title', 'abstract', 'authors', 'language', 'grants', 'publication_types', 'electronic_publication_date', 'place_of_publication', 'journal_abbreviated', 'journal', 'nlm_journal_id', 'descriptors', 'pmcid', 'keywords', 'conflict_of_interest', 'received_time', 'revised_time', 'accepted_time', 'pubmed_time', 'medline_time', 'entrez_time', 'pii', 'doi', 'publication_status', 'print_issn', 'pages'])
complete_nbibtable.to_csv(db_path+'/complete_nbibtable.csv')

time_finish = datetime.now()
time_batch = time_finish - time_start
time_batch_str = str(time_batch).split('.')[0]
print('gene batch:', len(genes))
print('time batch:', time_batch_str)
print('runtime/gene:', time_batch/len(genes))
print('request_counter:', request_counter,' (limit: 10.000/day)')
gene_counter += len(genes)
print('requests/gene:', request_counter/gene_counter)
print(time_finish)

### notes:
# LIMITS PubMed Programming Utilities (PMU)
# 10 requests/second
# 10,000 requests/day

Start at  2024-03-15 11:45:49.383531
KRTAP4-8_runtime: 0:00:05.418950
CCL3_runtime: 0:00:04.962459
TAS2R50_runtime: 0:00:04.964937
MT-ND3_runtime: 0:00:05.391834
SIRPG_runtime: 0:00:05.396164
gene batch: 5
time batch: 0:00:26
runtime/gene: 0:00:05.296127
request_counter: 10  (limit: 10.000/day)
requests/gene: 2.0
2024-03-15 11:46:15.864166


In [21]:
print('grpm report:')
display(pd.read_csv(db_path+'/GRPM_report.csv', index_col=0).T)
print('grpm table genes:', len(pd.read_csv(db_path+'/grpm_table_output.csv').gene.drop_duplicates()))

print('\nnbib table:')
display(pd.read_csv(db_path+'/complete_nbibtable.csv',index_col=0))


grpm report:


,ncbi_dbsnp,lit2_variant,lit2_variant_norsid,lit2_rsid,lit2_rsid_plus1,lit1_rsid,lit1_rsid_pmid_plus1,lit1_pmid,lit1_pmid_pmid_plus1,pubmed_pmid_query,...,pubmed_mesh_qualifier_major,pubmed_mesh,rsid_pmid10,rsid_pmid50,rsid_pmid100,top10mesh_all,top10rsid_all,pubmed_runtime,total_runtime,time_stamp
TAP2,na,416,35,363,193,0,1,2,2,2,...,28,26,0,0,0,"['Adolescent', 'Adult', 'Humans', 'Genome-Wide...",['rs9501224'],0:00:02,0:00:05,2024-03-15 11:05:29.231491
PRODH,na,358,72,262,133,0,1,9,9,9,...,87,66,0,0,0,"['Humans', 'Genome-Wide Association Study', 'G...",['rs9618419'],0:00:02,0:00:05,2024-03-15 11:05:34.818106
MFHAS1,na,612,7,604,250,0,1,2,2,2,...,36,29,0,0,0,"['Humans', 'Adaptation, Physiological', 'Heel'...",['rs9942753'],0:00:02,0:00:05,2024-03-15 11:05:40.187241
GNL1,na,153,6,147,52,0,1,2,2,2,...,29,25,0,0,0,"['Humans', 'Male', 'Isocitrate Dehydrogenase',...",['rs9468787'],0:00:02,0:00:04,2024-03-15 11:05:44.815576
ABCB11,na,1012,163,726,359,0,1,2,2,2,...,18,17,0,0,0,"['ATP Binding Cassette Transporter, Subfamily ...",['rs979738325'],0:00:02,0:00:05,2024-03-15 11:05:49.994041
WFDC10A,na,29,0,29,15,0,1,4,4,4,...,72,59,0,0,0,"['Humans', 'Young Adult', 'Polymorphism, Singl...",['rs80157014'],0:00:02,0:00:04,2024-03-15 11:05:59.741341
PCSK2,na,621,28,579,259,0,1,4,4,4,...,51,40,0,0,0,"['Humans', 'Genome-Wide Association Study', 'G...",['rs981662'],0:00:02,0:00:05,2024-03-15 11:06:05.006056
GART,na,292,38,248,100,0,1,5,5,5,...,69,53,0,0,0,"['Humans', 'Male', 'Gene Expression Profiling'...",['rs9636610'],0:00:02,0:00:04,2024-03-15 11:06:09.594079
LILRB1,na,581,27,535,251,0,1,2,2,2,...,36,28,0,0,0,"['Animals', 'Cell Line, Tumor', 'Signal Transd...",['rs995680547'],0:00:02,0:00:05,2024-03-15 11:06:14.936516
RPS4Y2,na,23,1,22,4,0,1,2,2,2,...,20,19,0,0,0,"['Humans', 'Male', 'Precision Medicine', 'Poly...",['rs746235827'],0:00:02,0:00:05,2024-03-15 11:17:23.046608


grpm table genes: 17

nbib table:


,gene,pubmed_id,citation_owner,nlm_status,last_revision_date,electronic_issn,linking_issn,journal_volume,journal_issue,publication_date,...,revised_time,accepted_time,pubmed_time,medline_time,entrez_time,pii,doi,publication_status,print_issn,pages
0,TAP2,28425483,NLM,MEDLINE,2023-11-12,2041-1723,2041-1723,8,NaN,2017 Apr 20,...,NaN,2017-01-27,2017-04-21 06:00:00,2018-11-10 06:00:00,2017-04-21 06:00:00,ncomms14828,10.1038/ncomms14828,epublish,NaN,14828
1,TAP2,26259071,NLM,MEDLINE,2019-04-23,2041-1723,2041-1723,6,NaN,2015 Aug 10,...,NaN,2015-07-01,2015-08-11 06:00:00,2016-04-28 06:00:00,2015-08-11 06:00:00,ncomms8971,10.1038/ncomms8971,epublish,NaN,7971
0,PRODH,36670122,NLM,MEDLINE,2023-12-07,2041-1723,2041-1723,14,1.0,2023 Jan 20,...,NaN,2022-12-21,2023-01-21 06:00:00,2023-01-25 06:00:00,2023-01-20 23:15:00,35724,10.1038/s41467-022-35724-1,epublish,NaN,342
1,PRODH,35449099,NLM,MEDLINE,2022-05-18,1479-5876,1479-5876,20,1.0,2022 Apr 21,...,NaN,2022-04-03,2022-04-23 06:00:00,2022-04-26 06:00:00,2022-04-22 05:13:00,3377,10.1186/s12967-022-03377-9,epublish,NaN,181
2,PRODH,27769252,NLM,MEDLINE,2022-12-07,1471-2490,1471-2490,16,1.0,2016 Oct 21,...,NaN,2016-10-14,2016-10-23 06:00:00,2017-03-03 06:00:00,2016-10-23 06:00:00,180,10.1186/s12894-016-0180-4,epublish,NaN,62
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,FCGBP,27397505,NLM,MEDLINE,2022-03-18 00:00:00,1097-4172,0092-8674,166,3,2016 Jul 28,...,2015-12-23 00:00:00,2016-06-03 00:00:00,2016-07-12 06:00:00,2016-12-15 06:00:00,2016-07-12 06:00:00,S0092-8674(16)30746-2,10.1016/j.cell.2016.06.017,ppublish,0092-8674,740-754
1,FCGBP,32579932,NLM,MEDLINE,2021-12-04 00:00:00,2211-1247,NaN,31,12,2020 Jun 23,...,2020-04-03 00:00:00,2020-06-02 00:00:00,2020-06-25 06:00:00,2021-04-29 06:00:00,2020-06-25 06:00:00,107806,10.1016/j.celrep.2020.107806,ppublish,NaN,107806
2,FCGBP,23042115,NLM,MEDLINE,2022-03-16 00:00:00,1546-1718,1061-4036,44,12,2012 Dec,...,NaN,2012-09-26 00:00:00,2012-10-09 06:00:00,2013-02-14 06:00:00,2012-10-09 06:00:00,ng.2446,10.1038/ng.2446,ppublish,1061-4036,1365-9
0,NOTCH4,27397505,NLM,MEDLINE,2022-03-18 00:00:00,1097-4172,0092-8674,166,3,2016 Jul 28,...,2015-12-23 00:00:00,2016-06-03 00:00:00,2016-07-12 06:00:00,2016-12-15 06:00:00,2016-07-12 06:00:00,S0092-8674(16)30746-2,10.1016/j.cell.2016.06.017,ppublish,0092-8674,740-754


# Show saved report

In [ ]:
# Visualize GRPM_report.csv
GRPM_report = pd.read_csv(db_path+'/GRPM_report.csv', index_col=0).transpose().reset_index().rename(columns={'index':'gene'})

repo_int_cols = ['lit2_variant', 'lit2_variant_norsid','lit2_rsid','lit2_rsid_plus1', 'lit1_rsid', 'lit1_rsid_pmid_plus1','lit1_pmid', 'lit1_pmid_pmid_plus1','pubmed_pmid_query',    'nbib_objects', 'nbib_objects_withdescriptors', 'pubmed_pmid', 'pubmed_pmid_withmesh', 'pubmed_pmidmesh','pubmed_mesh_qualifier_major','pubmed_mesh', 'rsid_pmid10','rsid_pmid50', 'rsid_pmid100' ]

GRPM_report[repo_int_cols] = GRPM_report[repo_int_cols].astype(int)

#display(GRPM_report_less.sort_values(by= 'matching_pmids',ascending=False))
GRPM_report.sort_values(by='lit1_pmid',ascending = False)

In [ ]:
# Show Bar Graph
GRPM_report_sort = GRPM_report.sort_values(by= 'pubmed_pmid',ascending=False)

x = GRPM_report_sort.gene.iloc[:40]
y = GRPM_report_sort['pubmed_pmid'].iloc[:40]
plt.figure(figsize=(4, 8))
plt.title('PMIDs in Dataset', loc='center',pad=10)

plt.barh(x,y)
plt.gca().invert_yaxis()
plt.tick_params(axis='x', which='both', top=True, bottom=False, labeltop=True, labelbottom=False)
#plt.xlabel('pmid count', position=(0.5, 1.08))
plt.ylabel('genes')
plt.xlabel('pmid count', position=(0.5, 1.08))
ax = plt.gca()
ax.xaxis.set_label_position('top')
#plt.savefig('PMID_filtered.png',dpi=300, bbox_inches = "tight")
plt.show()

# Debugging Code Block

## debugging: litvar data

In [ ]:
# if stucked check 'data' variable:
# replace manually malformed lines
data

## debugging: nbib

In [ ]:
#NBIB PROBLEM SOLVER---------------------------------
# replace malformed lines
fullnbib= fullnbib.replace('2007/09/31','2007/09/30') # <= some dates are mispelled in pubmed
ref = nbib.read(fullnbib)
dfbib = pd.DataFrame(ref)
dfbib

NBIB PROBLEM SOLVER [History] ---------------------------------
with open('nbib report '+gene+'.txt', 'w', encoding='utf-8') as file:
    file.write(fullnbib)
with open('nbib report '+gene+'_FIXED.txt', 'r', encoding='utf-8') as file:
    fullnbib = file.read() # --> not work

# extras

## Eutils: get study type

In [ ]:
check_and_install_module('biopython')
from Bio import Entrez
Entrez.email = "your_email@example.com"

In [ ]:
### EUTILS GET STUDY TYPE MODULE
#https://biopython.org/docs/1.76/api/Bio.Entrez.html
def get_study_type(pmids):
    Entrez.email = 'your_email@your_domain.com'
    handle = Entrez.esummary(db='pubmed', id=','.join(pmids), retmode='xml')
    records = Entrez.parse(handle)
    study_types = []
    for record in records:
        article_types = record['PubTypeList']
        if 'Randomized Controlled Trial' in article_types:
            study_types.append('Randomized Controlled Trial')
        elif 'Controlled Clinical Trial' in article_types:
            study_types.append('Controlled Clinical Trial')
        elif 'Cohort Studies' in article_types:
            study_types.append('Cohort Study')
        elif 'Case-Control Studies' in article_types:
            study_types.append('Case-Control Study')
        elif 'Review' in article_types:
            study_types.append('Review')
        elif 'Clinical Trial' in article_types:
            study_types.append('Clinical Trial')
        elif 'Meta-Analysis' in article_types:
            study_types.append('Meta-Analysis')
        elif 'Multicenter Study' in article_types:
            study_types.append('Multicenter Study')
        else:
            study_types.append('Unknown')
    return study_types

pmidlist = list(pmidmesh['pmids'].drop_duplicates())
genepmids_str = list(map(str, pmidlist))
study_type = get_study_type(genepmids_str)
pmids_studytype = pd.DataFrame(list(zip(genepmids_str, study_type)), columns=[gene + '_PMID', 'study type'])
request_counter += 1

#study type count:
pmids_studytype_count = pmids_studytype.groupby('study type').describe().reset_index()
pmids_studytype_count.columns = pmids_studytype_count.columns.to_flat_index()
new_column_names = ['study_type', 'pmid-count', 'pmid-unique', 'pmid-top', 'pmid-freq']
pmids_studytype_count.columns = new_column_names
pmids_studytype_countsort = pmids_studytype_count.sort_values(by='pmid-count', ascending=False)